# Reconstruction Tutorial - Irregular Constellations
This notebook demonstrates how to perform a reconstruction for an irregular constellation. In general, the idea is to create or retrieve the trajectory, assemble the trajectories into a single one, and then give that to the reconstruction tool with no satellife offsets. For the example here, the trajectories for four DMSP satellites are retrieved during a storm in March of 2013. The reconstruction is performed using WAM-IPE data for the same storm obtained from https://ccmc.gsfc.nasa.gov/RoR_WWW/output_files/KAMODO_DEMO/.

In [ ]:
# What times are in the data chosen?
import kamodo_ccmc.flythrough.model_wrapper as MW
model, file_dir = 'WAMIPE', 'D:/WAMIPE/storm_201303/'
times = MW.File_Times(model, file_dir)
times, times[0].timestamp(), times[1].timestamp()

The DMSP satellites flying during the March 2013 storm are dmspf15, dmspf16, dmspf17, and dmspf18. The trajectories for these satellites will be retrieved using the time boundaries from the chosen model data. Choose the coordinate system for input_coord you wish to perform the reconstruction in. Note that the coordinate conversion breaks in SpacePy for GSE -> GDZ due to a data size change caused by unknown reasons (https://github.com/spacepy/spacepy/issues/673). Given that most ITM model are aligned with the GDZ coordinate system, this will cause the code to break. For now, avoid reconstructing in GSE coordinates for ITM models aligned with the GDZ coordinate system.

In [ ]:
# Choose a variable from the model data chosen.
MW.Variable_Search('', model, file_dir)

In [ ]:
# Import function to retrieve the DMSP trajectories from the SSCWeb.
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
# Typical coordinates possible through SSCWeb are GEO, GSE, SM, and GSM (all cartesian and in R_E).
input_coord = 'GEO'
traj_dict15, coord_type = SF.SatelliteTrajectory('dmspf15', times[0].timestamp(), times[1].timestamp(), coord_type=input_coord)
traj_dict16, coord_type = SF.SatelliteTrajectory('dmspf16', times[0].timestamp(), times[1].timestamp(), coord_type=input_coord)
traj_dict17, coord_type = SF.SatelliteTrajectory('dmspf17', times[0].timestamp(), times[1].timestamp(), coord_type=input_coord)
traj_dict18, coord_type = SF.SatelliteTrajectory('dmspf18', times[0].timestamp(), times[1].timestamp(), coord_type=input_coord)

In [ ]:
# Assemble the trajectories into one input per coordinate
import numpy as np
time, X, Y, Z = traj_dict15['sat_time'], traj_dict15['c1'], traj_dict15['c2'], traj_dict15['c3']
for item in [traj_dict16, traj_dict17, traj_dict18]:
    time = np.append(time, item['sat_time'])
    X = np.append(X, item['c1'])
    Y = np.append(Y, item['c2'])
    Z = np.append(Z, item['c3'])

In [ ]:
# Plot the combined trajectory to have a look
from kamodo_ccmc.flythrough.plots import SatPlot4D
hrs = (time - time.min())/3600.
SatPlot4D('utc_time', time, X, Y, Z, hrs, 'hr', input_coord, 'car', input_coord, 'all', 'DMSP15-18', body='black', type='3D')

In [ ]:
# Convert coordinates to system desired for reconstruction to take place in.
# The trajectories were retrieved in GEO cartesian, and are converted to GEO spherical below.
# See Trajectory_Coords_Plots notebook for more details.
from kamodo_ccmc.flythrough.utils import ConvertCoord
c1, c2, c3, units = ConvertCoord(time, X, Y, Z, input_coord, 'car', input_coord, 'sph') 

In [ ]:
from kamodo_ccmc.flythrough.Reconstruction_v0 import RECON
help(RECON)

Prepare inputs and settings for the reconstruction execution. The default offset values are zero, meaning that only one satellite is flown through by default. This is what we want here because the trajectories of all four satellites have been combined into one object

In [ ]:
# Choose inputs.
variable_name = 'TEC'  # from chosen files above
recon_dimensions = 'c1c2'  # Longitude vs Latitude reconstruction for spherical coordinates
recon_option = 'UnMod_AvgDSlice'

# Choose the grid resolution of reconstruction. The finer the resolution, the longer the program takes to run and 
#   the more 'holes' you will see in the reconstructed plot. Physically, these should be set to the instrument's 
#   field of view in the units of the input coordinate system (e.g. degrees for longitude and latitude, seconds for
#   time, etc).
dx, dy = 4., 2.  # Since recon_dimensions='c1c1', dx is resolution in longitude, and dy is the resolution in latitude.
d1, d2 = 3600., 1.0  # time and vertical

In [ ]:
# Run the reconstruction.
recon = RECON(model, variable_name, file_dir, time, c1, c2, c3, input_coord, 'sph',
              recon_option, recon_dimensions, dx=dx, dy=dy, d1=d1, d2=d2)
recon

In [ ]:
# Show the reconstructed plot.
# Note the plots shown are given in the coordinate system associated with the input trajectory.
# Any gaps in the reconstructed plot indicate gaps in the satellite coverage of the plot grid chosen (dx and dy above),
#   and also depend on the constellation arrangement.

# recon.x and recon.y are the x and y grids of the plots.
recon.plot(TEC=dict(Lon=recon.x, Lat=recon.y))

In [ ]:
# Nicer version of same plot
fig = recon.plot(TEC=dict(Lon=recon.x, Lat=recon.y))
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# Show the corresponding model data
recon.plot(TEC_model=dict(Lon=recon.x, Lat=recon.y))

In [ ]:
# Nicer version of same plot
fig = recon.plot(TEC_model=dict(Lon=recon.x, Lat=recon.y))
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# Show the percent difference between the two. A percent difference of zero is an exact match.
# How well the reconstructed plot matches the model plot not only depends on the constellation arrangement,
#  but also on the reconstruction method chosen. The 'AvgMod_...' options typically result in the better
#  matches, but are not physically representative of what the constellation will 'see' in real data because
#  the two non-reconstructed dimensions are ignored in the input satellite trajectory (e.g. an average value 
#  for both height and time are used instead of the full range for a Lon-Lat reconstruction). The unmodified 
#  options ('Unmod_...') are thus recommended as the more physical comparison because the full set of input 
#  trajectory values are used.
recon.plot(PercentDiff=dict(Lon=recon.x,Lat=recon.y))

In [ ]:
# Nicer version of same plot
fig = recon.plot(PercentDiff=dict(Lon=recon.x, Lat=recon.y))
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# Retrieve the percent difference data values and show in a histogram, ignoring NaN values.
# Some extra logic is required to automatically enforce bins of width 2%
import numpy as np
import matplotlib.pyplot as plt

pdiff_data = recon.PercentDiff()
data_min, data_max = np.floor(np.nanmin(pdiff_data)), np.ceil(np.nanmax(pdiff_data))
num_bins = int((data_max-data_min)/2.)
if num_bins < 5:
    num_bins = 20
hist, edges, patches = plt.hist(np.ravel(pdiff_data), range=(data_min, data_max), bins=num_bins)
plt.xlabel('Percent Difference')
plt.ylabel('Frequency')